Notebook para hacer el finetuning de un transformer (distilbert pero probaremos otros). Solo estamos teniendo en cuenta el texto de la noticia.

Habrá que tener en cuenta el resto de características para ver cómo mejora. Para los estados y partido afiliado, puede ser útil relacionarlo con esto https://www.adfontesmedia.com/ de cara a la presrentación de resultados.

In [1]:
from nlp import load_dataset
dataset = load_dataset('csv', data_files={'train': 'oversampled_balanced_train.csv'})
test = load_dataset('csv', data_files={'test': 'test_nolabel.csv'})

Using custom data configuration default
Using custom data configuration default


In [2]:
dataset = dataset['train'].train_test_split(test_size=0.1)

In [3]:
dataset


{'train': Dataset(features: {'': Value(dtype='int64', id=None), 'id': Value(dtype='string', id=None), 'statement': Value(dtype='string', id=None), 'subject': Value(dtype='string', id=None), 'speaker': Value(dtype='string', id=None), 'speaker_job': Value(dtype='string', id=None), 'state_info': Value(dtype='string', id=None), 'party_affiliation': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None)}, num_rows: 10431),
 'test': Dataset(features: {'': Value(dtype='int64', id=None), 'id': Value(dtype='string', id=None), 'statement': Value(dtype='string', id=None), 'subject': Value(dtype='string', id=None), 'speaker': Value(dtype='string', id=None), 'speaker_job': Value(dtype='string', id=None), 'state_info': Value(dtype='string', id=None), 'party_affiliation': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None)}, num_rows: 1159)}

In [4]:
test

{'test': Dataset(features: {'id': Value(dtype='string', id=None), 'statement': Value(dtype='string', id=None), 'subject': Value(dtype='string', id=None), 'speaker': Value(dtype='string', id=None), 'speaker_job': Value(dtype='string', id=None), 'state_info': Value(dtype='string', id=None), 'party_affiliation': Value(dtype='string', id=None)}, num_rows: 3836)}

In [5]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch
import pandas as pd

import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [6]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['label'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

class TestDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average=None)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [7]:
models = ['xlnet-base-cased'] #'bert-base-cased', 'roberta-base', 'xlnet-base-cased'
LR = 2e-5
EPOCHS = 5
BATCH_SIZE = 4
MAX_TRAINING_EXAMPLES = -1 # set this to -1 if you want to use the whole training set

for model in models:
    for epoch in range(EPOCHS):

        if epoch == 0:
            MODEL = model # use this to finetune the language model

            tokenizer = AutoTokenizer.from_pretrained(MODEL, use_fast=True)

            train_encodings = tokenizer(dataset['train']['statement'], truncation=True, padding=True)
            val_encodings = tokenizer(dataset['test']['statement'], truncation=True, padding=True)
            test_encodings = tokenizer(test['test']['statement'], truncation=True, padding=True)



            train_dataset = MyDataset(train_encodings, dataset['train']['label'])
            val_dataset = MyDataset(val_encodings, dataset['test']['label'])
            test_dataset = TestDataset(test_encodings)





            training_args = TrainingArguments(
                output_dir='./results',                   # output directory
                num_train_epochs=1,                  # total number of training epochs
                per_device_train_batch_size=BATCH_SIZE,   # batch size per device during training
                per_device_eval_batch_size=BATCH_SIZE,    # batch size for evaluation
                warmup_steps=100,                         # number of warmup steps for learning rate scheduler
                weight_decay=0.01,                        # strength of weight decay
                logging_dir='./logs',                     # directory for storing logs
                logging_steps=10,                         # when to print log
                load_best_model_at_end=True,              # load or not best model at the end
            )

            num_labels = len(set(dataset['train']['label']))
            print(f'Num labels: {num_labels}')
            model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=num_labels)

            trainer = Trainer(
                model=model,                              # the instantiated 🤗 Transformers model to be trained
                args=training_args,                       # training arguments, defined above
                train_dataset=train_dataset,              # training dataset
                eval_dataset=val_dataset,                 # evaluation dataset 
                compute_metrics = compute_metrics
            )
        trainer.train()

        trainer.save_model(f"./results/best_model_{MODEL}_{epoch+1}_epochs_overbalanced") # save best model
        
        
        print(f'=============== Predicting ======================')
        PredictionOutput = trainer.predict(test_dataset)

        result = np.argmax(PredictionOutput.predictions, axis=1)

        ids = test['test']['id']

        df = pd.DataFrame(ids, columns=['id'])

        df['label'] = result
        df

        df.to_csv(f'submissions/submission_{MODEL}_{epoch+1}_epochs_overbalanced.csv', index=False)
        
        try:
            print(f'=============== Evaluating {MODEL} ======================')
            print(trainer.evaluate())
        except:
            print('Exception when evaluating')
            continue

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Num labels: 2


Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

Step,Training Loss
10,0.770100
20,0.769400
30,0.714000
40,0.676300
50,0.726700
60,0.690900
70,0.739700
80,0.720100
90,0.731500
100,0.712300


=============== Predicting ======================


...

=============== Predicting ======================
=============== Evaluating xlnet-base-cased ======================
Exception when evaluating


Step,Training Loss
10,0.526500
20,0.670800
30,0.574100
40,0.641300
50,0.621900
60,0.669900
70,0.621200
80,0.635500
90,0.651400
100,0.628300


=============== Predicting ======================
=============== Evaluating xlnet-base-cased ======================
Exception when evaluating


Step,Training Loss
10,0.667400
20,0.601400
30,0.541100
40,0.615900
50,0.592700
60,0.553900
70,0.597000
80,0.633200
90,0.614600
100,0.697500


=============== Predicting ======================
=============== Evaluating xlnet-base-cased ======================
Exception when evaluating


In [8]:
models = ['bert-base-cased','roberta-base',] #'bert-base-cased', 'roberta-base', 'xlnet-base-cased'
LR = 2e-5
EPOCHS = 5
BATCH_SIZE = 8
MAX_TRAINING_EXAMPLES = -1 # set this to -1 if you want to use the whole training set

for model in models:
    for epoch in range(EPOCHS):

        if epoch == 0:
            MODEL = model # use this to finetune the language model

            tokenizer = AutoTokenizer.from_pretrained(MODEL, use_fast=True)

            train_encodings = tokenizer(dataset['train']['statement'], truncation=True, padding=True)
            val_encodings = tokenizer(dataset['test']['statement'], truncation=True, padding=True)
            test_encodings = tokenizer(test['test']['statement'], truncation=True, padding=True)



            train_dataset = MyDataset(train_encodings, dataset['train']['label'])
            val_dataset = MyDataset(val_encodings, dataset['test']['label'])
            test_dataset = TestDataset(test_encodings)





            training_args = TrainingArguments(
                output_dir='./results',                   # output directory
                num_train_epochs=1,                  # total number of training epochs
                per_device_train_batch_size=BATCH_SIZE,   # batch size per device during training
                per_device_eval_batch_size=BATCH_SIZE,    # batch size for evaluation
                warmup_steps=100,                         # number of warmup steps for learning rate scheduler
                weight_decay=0.01,                        # strength of weight decay
                logging_dir='./logs',                     # directory for storing logs
                logging_steps=10,                         # when to print log
                load_best_model_at_end=True,              # load or not best model at the end
            )

            num_labels = len(set(dataset['train']['label']))
            print(f'Num labels: {num_labels}')
            model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=num_labels)

            trainer = Trainer(
                model=model,                              # the instantiated 🤗 Transformers model to be trained
                args=training_args,                       # training arguments, defined above
                train_dataset=train_dataset,              # training dataset
                eval_dataset=val_dataset,                 # evaluation dataset 
                compute_metrics = compute_metrics
            )
        trainer.train()

        trainer.save_model(f"./results/best_model_{MODEL}_{epoch+1}_epochs_overbalanced") # save best model
        try:
            print(f'=============== Evaluating {MODEL} ======================')
            print(trainer.evaluate())
        except:
            print('Exception when evaluating')
            continue
        
        print(f'=============== Predicting ======================')
        PredictionOutput = trainer.predict(test_dataset)

        result = np.argmax(PredictionOutput.predictions, axis=1)

        ids = test['test']['id']

        df = pd.DataFrame(ids, columns=['id'])

        df['label'] = result
        df

        df.to_csv(f'submissions/submission_{MODEL}_{epoch+1}_epochs_overbalanced.csv', index=False)
        

Num labels: 2


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Step,Training Loss
10,0.734400
20,0.799800
30,0.694500
40,0.680700
50,0.712200
60,0.703200
70,0.711900
80,0.720000
90,0.683000
100,0.690100


=============== Evaluating bert-base-cased ======================


{'eval_loss': 0.6033388376235962, 'eval_accuracy': 0.6824849007765315, 'eval_f1': array([0.65607477, 0.70512821]), 'eval_precision': array([0.68554688, 0.68006182]), 'eval_recall': array([0.62903226, 0.73211314]), 'eval_runtime': 42.6089, 'eval_samples_per_second': 27.201, 'epoch': 1.0, 'eval_mem_cpu_alloc_delta': 65536, 'eval_mem_gpu_alloc_delta': 0, 'eval_mem_cpu_peaked_delta': 16384, 'eval_mem_gpu_peaked_delta': 26560512}
=============== Predicting ======================


...

Step,Training Loss
10,0.595200
20,0.599400
30,0.649000
40,0.608200
50,0.573100
60,0.602100
70,0.590000
80,0.634100
90,0.648600
100,0.633000


=============== Evaluating roberta-base ======================
{'eval_loss': 0.6269557476043701, 'eval_accuracy': 0.6600517687661778, 'eval_f1': array([0.65798611, 0.66209262]), 'eval_precision': array([0.63804714, 0.68318584]), 'eval_recall': array([0.67921147, 0.6422629 ]), 'eval_runtime': 43.912, 'eval_samples_per_second': 26.394, 'epoch': 1.0, 'eval_mem_cpu_alloc_delta': 0, 'eval_mem_gpu_alloc_delta': 0, 'eval_mem_cpu_peaked_delta': 81920, 'eval_mem_gpu_peaked_delta': 26108928}
=============== Predicting ======================


/opt/conda/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:64: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
10,0.600500
20,0.591900
30,0.642700
40,0.598800
50,0.660800
60,0.628600
70,0.629100
80,0.629600
90,0.559800
100,0.609300


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

